In [1]:
DATA_DIR = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots\terrain_data'

In [7]:
%cd "C:\Users\User\OneDrive\Documents\GitHub\image-classification-yolov8"

C:\Users\User\OneDrive\Documents\GitHub\image-classification-yolov8


In [ ]:
%pip install -r requirements.txt

In [2]:
import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n-cls.pt")  # load a pretained model

# Use the model
results = model.train(data=DATA_DIR, epochs=10, imgsz=64)  # train the model


100%|██████████| 5.28M/5.28M [00:01<00:00, 4.04MB/s]
New https://pypi.org/project/ultralytics/8.0.221 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.58  Python-3.11.6 torch-2.1.1+cpu CPU
yolo\engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots\terrain_data, epochs=10, patience=50, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_strid

In [3]:
# metrics = model.val(os.path.join(DATA_DIR, 'test'))  # calculate metrics

# print(metrics)

In [5]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# import torch
# from ultralytics import YOLO  # Make sure to install Ultralytics YOLO package
# from torchvision import transforms
# from torch.utils.data import DataLoader
# import sys


# def test_model_accuracy(model_path, test_data_path, batch_size=16, num_classes=80):
#     # Load the trained model
#     model = YOLO(model_path)

#     # Prepare the test dataset
#     transform = transforms.Compose([
#         transforms.ToTensor(),
#         # Add any other transformations you used during training
#     ])
#     test_dataset = CustomImageDataset(test_data_path, transform=transform)  # Replace with your dataset
#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#     # Switch model to evaluation mode
#     model.eval()

#     # Variables to store performance metrics
#     total, correct = 0, 0

#     with torch.no_grad():
#         for images, labels in test_loader:
#             outputs = model(images)
#             # Calculate accuracy
#             # Note: You need to modify this part based on how your labels and outputs are structured
#             # This is a placeholder logic for accuracy calculation
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     accuracy = 100 * correct / total
#     return accuracy

# # Example usage
# model_path = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\ModelTraining\runs\detect\terrain_train\runs\classify\train\weights\best.pt'
# test_data_path = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots\terrain_data\test'
# accuracy = test_model_accuracy(model_path, test_data_path)
# print(f"Model Accuracy: {accuracy}%")


In [8]:
# test_model = YOLO(model_path) 

# test_model.predict()

In [ ]:
test_model = YOLO(model_path)   

test_model.val()